# Discret logarithmic problème

## Dans le groupe additif $( \mathbb{Z}_p, + )$

étant donné un nombre premier \(p\), un générateur \(g\) du groupe additif $\mathbb{Z}_p$, et un élément $y \in \mathbb{Z}_p$, trouver $x$ tel que :
    $a \cdot x \equiv b \pmod{p}$

In [1]:
import sympy as sp
import numpy as np
import math

# variables globales
variables=[]  # tout les variables créés
dictionaire_auxiliary_variable={}
res=1
x=sp.Add()
y=sp.Add()
k=sp.Add()

# Créer un symbole à partir d'un nom en string
def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

# les varaiables sont sort par la lettre puis un nombre: "x12" -> "x" 12
def sort_key(variable):
    name = str(variable)
    # Split le nom du variable aux deux parties: letter & number
    letter = name[0]
    number = int(name[1:])
    return (letter, number)

def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term

# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]

    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1
    else:
        x_4 = key

    expr_reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = rest_expr * expr_reduct_formula
    return sp.expand(expr_reduct)

def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste)
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction: # récursive
        return reduct_demension_expression(new_expr)
    else:
        return new_expr

def get_reduct_expression(a,b,p):
    expr = define_function(a,b,p)
    print(f" define_function : {expr}")
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
    return reduct_demension_expr

def transfers_qubo2Ising(reduct_demension_expr):
    subs_relations = {(variable, ( variable + 1 ) /2) for variable in variables}

    # Après la substitution, variables dans l'expression sont des variables d'Ising dans {-1,+1}
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

import dimod
from dwave.preprocessing.composites import SpinReversalTransformComposite

# Compose the sampler
base_sampler = dimod.ExactSolver()
composed_sampler = SpinReversalTransformComposite(base_sampler)

def get_solver_parameter(expr_ising):
    linear = {}
    quadratic ={}
    offset = 0

    for term in expr_ising.args:
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]
        if len(term.free_symbols) ==0:
            offset = coef
        if len(term.free_symbols) ==1:
            linear[variables_liste[0].name] = coef
        elif len(term.free_symbols) ==2:
            quadratic[(variables_liste[0].name,variables_liste[1].name)] = coef

    return (linear,quadratic,offset)

def get_solution(expr_ising):
    (linear,quadratic,offset) = get_solver_parameter(expr_ising)
    response = composed_sampler.sample_ising(linear, quadratic)  #response.data() return (solution, energy, num_occurrences)
    solution = next(response.data())[0]
    return solution

### définir fonction
$(x \cdot a \bmod p ) + (-b \bmod p) - k\cdot p = 0 $

In [2]:
# set variable en binaire qui est dans {0,...,p-1}

# exemple : x = u1 + 2u2 + 4u3 + ... + 2^(n-2)un-1 + (p-2^(n-1)+1)un
def set_x_expression(n,p):  # n est length de x en binaire
    # len(x) = len(p)
    expr_x = 0
    # de u1 à un-1
    for i in range(1,n):
        var_name = ('u' + str(i))
        var = create_variables(var_name)
        expr_x = 2**(i-1) * var + expr_x
    # un
    expr_x = expr_x + (p-2**(n-1)+1) * create_variables('u' + str(n))
    return expr_x

def set_k_expression(n):  # l est length de k en binaire
    l = math.floor(math.log2(n)) + 1
    expr_k = 0
    # de u1 à un-1
    for i in range(1,l):
        var_name = ('k' + str(i))
        var = create_variables(var_name)
        expr_k = 2**(i-1) * var + expr_k
    # un
    expr_k = expr_k + (n-2**(l-1)+1) * create_variables('k' + str(l))
    return expr_k

def define_function(a, b, p):
    # set variables for x
    n = math.ceil(math.log2(p))  # len de x == len de p
    global x
    x = set_x_expression(n,p)

    #set variables for k
    global k
    k = set_k_expression(n)

    # set expression
    expr = x * sp.Mod(a,p)
    expr = expr + sp.Mod(-b,p)
    expr = expr - (k * p)
    expr = (expr) **2
    expr = sp.expand(expr)
    return expr

In [3]:
expr = define_function(2,3,5)
expr

25*k1**2 + 100*k1*k2 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 - 20*k1 + 100*k2**2 - 40*k2*u1 - 80*k2*u2 - 80*k2*u3 - 40*k2 + 4*u1**2 + 16*u1*u2 + 16*u1*u3 + 8*u1 + 16*u2**2 + 32*u2*u3 + 16*u2 + 16*u3**2 + 16*u3 + 4

In [4]:
expr_reduct_power = reduct_variable_power_function(expr)
expr_reduct_power

100*k1*k2 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 + 5*k1 - 40*k2*u1 - 80*k2*u2 - 80*k2*u3 + 60*k2 + 16*u1*u2 + 16*u1*u3 + 12*u1 + 32*u2*u3 + 32*u2 + 32*u3 + 4

In [5]:
reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
reduct_demension_expr

100*k1*k2 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 + 5*k1 - 40*k2*u1 - 80*k2*u2 - 80*k2*u3 + 60*k2 + 16*u1*u2 + 16*u1*u3 + 12*u1 + 32*u2*u3 + 32*u2 + 32*u3 + 4

## On obtient l'expression sous forme de QUBO, ensuit on peut la transferer sous forme de modèle d'Ising

In [6]:
expr_ising = transfers_qubo2Ising(reduct_demension_expr)
expr_ising

25*k1*k2 - 5*k1*u1 - 10*k1*u2 - 10*k1*u3 + 5*k1/2 - 10*k2*u1 - 20*k2*u2 - 20*k2*u3 + 5*k2 + 4*u1*u2 + 4*u1*u3 - u1 + 8*u2*u3 - 2*u2 - 2*u3 + 81/2

# Find solution avec dimon bibilothèque

In [7]:
solution = get_solution(expr_ising)
solution

{'k2': np.int8(1),
 'u2': np.int8(1),
 'u3': np.int8(1),
 'k1': np.int8(-1),
 'u1': np.int8(-1)}

In [8]:
def calcule_x(solution,x,k):
    dic_varialbe_value = {}
    for var in variables:
        if(solution[var.name] == np.int8(1)):
            dic_varialbe_value[var] = int(1)
        else:
            dic_varialbe_value[var] = int(0)
    x_value = x.subs(dic_varialbe_value)
    k_value = k.subs(dic_varialbe_value)
    
    return (x_value.evalf(),k_value.evalf())


def factorization(a, b, p):
    global variables, dictionaire_auxiliary_variable, res,x,k
    variables = []
    dictionaire_auxiliary_variable = {}
    res = 1
    x= sp.Add()
    k= sp.Add()


    reduct_demension_expr = get_reduct_expression(a,b,p)
    expr_ising = transfers_qubo2Ising(reduct_demension_expr)
    solution = get_solution(expr_ising)
    (x_val,k_val) = calcule_x(solution,x,k)
    x_int = int(x_val)
    k_int = int(k_val)
    print(f" k: {k_int}")
    print(f"The solution of {a} * x = {b} mod {p} is : x = {x_int}.")

In [9]:
a=2
b=3
p=5
factorization(a, b, p)

 define_function : 25*k1**2 + 100*k1*k2 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 - 20*k1 + 100*k2**2 - 40*k2*u1 - 80*k2*u2 - 80*k2*u3 - 40*k2 + 4*u1**2 + 16*u1*u2 + 16*u1*u3 + 8*u1 + 16*u2**2 + 32*u2*u3 + 16*u2 + 16*u3**2 + 16*u3 + 4
 k: 2
The solution of 2 * x = 3 mod 5 is : x = 4.


In [10]:
a=5
b=5
p=7
factorization(a, b, p)

 define_function : 49*k1**2 + 196*k1*k2 - 70*k1*u1 - 140*k1*u2 - 280*k1*u3 - 28*k1 + 196*k2**2 - 140*k2*u1 - 280*k2*u2 - 560*k2*u3 - 56*k2 + 25*u1**2 + 100*u1*u2 + 200*u1*u3 + 20*u1 + 100*u2**2 + 400*u2*u3 + 40*u2 + 400*u3**2 + 80*u3 + 4
 k: 1
The solution of 5 * x = 5 mod 7 is : x = 1.


# Dans le groupe multiplicative $( \mathbb{Z}_p, \cdot )$

## Brutal approach
On define le DLP comme ceci:
    $g^y = h $

Ensuite, dans $\mathbb{F}_p, g,h \in \{1, \cdots , p-1\}, y \in \{1, \cdots, \text{Ord}(g)\}$


$g^y \equiv h \pmod{p}$


#### Defint y:
set $m$ est le nombre de bits de $y$ = bitlength of ord(g).

$y$ peux être définit comme ceci:
    $ y = 2^{m - 1} u_m + \cdots + 2u_2 + u_1$

union de equation 6 et 8, on obtient equation 13

## Fonctions basiques:
1. calculer ord(g) dans p
2. définir valeur unconnu en l'expression composé des variables binaires

In [11]:
def calculate_order(g, p):
    # ord(g): le min k qui satisfait g^k = 1 mod p
    if g <= 0 or g >= p:
        print(f"error：g should in { 1, p-1} ")
        return None

    order = 1
    current = g % p

    while current != 1:
        # g^(order+1) mod p
        current = (current * g) % p
        order += 1

        if order > p-1:
            print(f"error：p is not prime or g and p are not prime")
            return None
    return order

def set_variable_expression(n,prefix):  # n is the length of variable and the prefix of each binary variable
    expr_var = 0
    # de u1 à un
    for i in range(1,n+1):
        var_name = (prefix + str(i))
        var = create_variables(var_name)
        expr_var = 2**(i-1) * var + expr_var
    return expr_var


# $g^y$
## Definir $g^y$ en forme de equation 6:

$g^y = g^{2^{m-1} \cdot u_m+\cdots+2u_2+u_1}  = g^{2^{m-1}} \cdots g^{2u_2}g^{u_1}$

In [12]:
def create_g_power_expression(expr_y,g):
    """
    g^y = g^(2^0*u₁ + 2^1*u₂+ ... + 2^(n-1)*uₙ)
    """

    g_power_y = g ** expr_y

    return g_power_y

def decompose_g_power(g_power_expr):
    """
    g^y = g^(2^0*u₁) * g^(2^1*u₂) * ... * g^(2^(n-1)*uₙ)
    """
    exponent = g_power_expr.as_base_exp()[1]
    base = g_power_expr.as_base_exp()[0]
    decomposed_expr = 1

    for term in exponent.args:
        decomposed_expr = decomposed_expr * base ** term
    return decomposed_expr

In [13]:
g=2
h=3
p=5

ord = calculate_order(g,p)
n = math.ceil(math.log2(ord))+1

expr_y = set_variable_expression(n,'u')
g_power_y = create_g_power_expression(expr_y,g)
print(f"expression of g^y : {g_power_y}")

decomposed_expr = decompose_g_power(g_power_y)
print(f"expression of decomposed g^y : {decomposed_expr}")

expression of g^y : 2**(u1 + 2*u2 + 4*u3)
expression of decomposed g^y : 2**u1*2**(2*u2)*2**(4*u3)


## Transforme en utilisant l'équation 8 de l'equation 6
transfert formule : $g^{2^{i-1} \cdot u_i} = 1+u_i(g^{2^{i-1}}-1)$

$g^y =(1 + u_m(g^{2^{m−1}}− 1))  \cdots (1 + u_2 (g^2 − 1)) \dot (1 + u_1 (g − 1))$


In [14]:
def transform_using_formula(decomposed_expr,p):
    """
    g^(2^(i-1) * u_i) = 1 + u_i * (g^(2^(i-1)) - 1 % p)
    """
    terms = []
    for element in decomposed_expr.args:
        ele = element.exp

        if ele.is_symbol:
            #u_1
            transformed_element = 1 + ele * (element.base - 1)
            terms.append(transformed_element)
        elif ele.is_Mul:
            [coeff, u_var] = ele.args
            #val = pow(int(element.base), int(coeff), int(p))
            #transformed_element = 1 + u_var * (val - 1)
            transformed_element = 1 + u_var * (element.base**(coeff) - 1)
            terms.append(transformed_element)
    return sp.Mul(*terms)

In [15]:
transformed_expr = transform_using_formula(decomposed_expr,p)
print(f"expression of transformed g^y: {transformed_expr}")

sp.expand(transformed_expr)

expression of transformed g^y: (u1 + 1)*(3*u2 + 1)*(15*u3 + 1)


45*u1*u2*u3 + 3*u1*u2 + 15*u1*u3 + u1 + 45*u2*u3 + 3*u2 + 15*u3 + 1

## f : linear form of $g^y$

$f = 1 + u_1(g − 1) + u_2(g^2 − 1) + u_1u_2(g − 1) (g^2 − 1)$

En ajoutant nouveau variables v_i:

$f = 1 + u_1(g − 1) + u_2(g^2 − 1) + v_1(g − 1) (g^2 − 1)$ avec $v_1 = u_1 * u_2$

In [16]:
def linaire_form_expr(transformed_expr):
    expr = sp.expand(transformed_expr)
    i = 1
    new_expr = expr
    changed = True

    while changed:
        changed = False
        for element in new_expr.args:
            if len(element.free_symbols) == 2:
                vars_list = list(element.free_symbols)
                new_var = create_variables("v"+str(i))
                product = sp.Mul(vars_list[0], vars_list[1])
                dictionaire_auxiliary_variable[new_var.name] = product
                new_expr = new_expr.subs(product, new_var)
                i += 1
                changed = True
    return new_expr

In [17]:
linaire_expr=linaire_form_expr(transformed_expr)
linaire_expr

u1 + 3*u2 + 15*u3 + 3*v1 + 15*v2 + 45*v3 + 45*v4 + 1

## Définir la fonction objective
$ (( f-h ) - (k \cdot p))^2 = 0 $

In [18]:
expr_k = set_variable_expression(n,'k')
expr_k

k1 + 2*k2 + 4*k3

In [19]:
expr = linaire_expr - h
expr = expr - (expr_k * p)
expr = (expr)**2
expr = sp.expand(expr)
expr

25*k1**2 + 100*k1*k2 + 200*k1*k3 - 10*k1*u1 - 30*k1*u2 - 150*k1*u3 - 30*k1*v1 - 150*k1*v2 - 450*k1*v3 - 450*k1*v4 + 20*k1 + 100*k2**2 + 400*k2*k3 - 20*k2*u1 - 60*k2*u2 - 300*k2*u3 - 60*k2*v1 - 300*k2*v2 - 900*k2*v3 - 900*k2*v4 + 40*k2 + 400*k3**2 - 40*k3*u1 - 120*k3*u2 - 600*k3*u3 - 120*k3*v1 - 600*k3*v2 - 1800*k3*v3 - 1800*k3*v4 + 80*k3 + u1**2 + 6*u1*u2 + 30*u1*u3 + 6*u1*v1 + 30*u1*v2 + 90*u1*v3 + 90*u1*v4 - 4*u1 + 9*u2**2 + 90*u2*u3 + 18*u2*v1 + 90*u2*v2 + 270*u2*v3 + 270*u2*v4 - 12*u2 + 225*u3**2 + 90*u3*v1 + 450*u3*v2 + 1350*u3*v3 + 1350*u3*v4 - 60*u3 + 9*v1**2 + 90*v1*v2 + 270*v1*v3 + 270*v1*v4 - 12*v1 + 225*v2**2 + 1350*v2*v3 + 1350*v2*v4 - 60*v2 + 2025*v3**2 + 4050*v3*v4 - 180*v3 + 2025*v4**2 - 180*v4 + 4

In [20]:
def define_function(g, h, p):
    # set variables for
    ord = calculate_order(g,p)
    if ord :
        n = math.ceil(math.log2(ord))+1
    else:
        print(f"ord({g}) is none ")
        return None

    global expr_y
    expr_y = set_variable_expression(n,'u')

    # g^y = transformed_expr
    g_power_expr = create_g_power_expression(expr_y,g)
    decomposed_expr = decompose_g_power(g_power_expr)
    transformed_expr = transform_using_formula(decomposed_expr,p)
    linaire_expr = linaire_form_expr(transformed_expr)
    print(f"linaire_expr : {linaire_expr}")

    #set variables for k
    global expr_k
    expr_k = set_variable_expression(n,'k')

    expr = linaire_expr - h
    expr = expr - (expr_k * p)
    expr = (expr)**2
    expr = sp.expand(expr)
    return expr

In [21]:
# reduction
expr_reduct_power = reduct_variable_power_function(expr)
expr_reduct_power

100*k1*k2 + 200*k1*k3 - 10*k1*u1 - 30*k1*u2 - 150*k1*u3 - 30*k1*v1 - 150*k1*v2 - 450*k1*v3 - 450*k1*v4 + 45*k1 + 400*k2*k3 - 20*k2*u1 - 60*k2*u2 - 300*k2*u3 - 60*k2*v1 - 300*k2*v2 - 900*k2*v3 - 900*k2*v4 + 140*k2 - 40*k3*u1 - 120*k3*u2 - 600*k3*u3 - 120*k3*v1 - 600*k3*v2 - 1800*k3*v3 - 1800*k3*v4 + 480*k3 + 6*u1*u2 + 30*u1*u3 + 6*u1*v1 + 30*u1*v2 + 90*u1*v3 + 90*u1*v4 - 3*u1 + 90*u2*u3 + 18*u2*v1 + 90*u2*v2 + 270*u2*v3 + 270*u2*v4 - 3*u2 + 90*u3*v1 + 450*u3*v2 + 1350*u3*v3 + 1350*u3*v4 + 165*u3 + 90*v1*v2 + 270*v1*v3 + 270*v1*v4 - 3*v1 + 1350*v2*v3 + 1350*v2*v4 + 165*v2 + 4050*v3*v4 + 1845*v3 + 1845*v4 + 4

In [22]:
reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
reduct_demension_expr

100*k1*k2 + 200*k1*k3 - 10*k1*u1 - 30*k1*u2 - 150*k1*u3 - 30*k1*v1 - 150*k1*v2 - 450*k1*v3 - 450*k1*v4 + 45*k1 + 400*k2*k3 - 20*k2*u1 - 60*k2*u2 - 300*k2*u3 - 60*k2*v1 - 300*k2*v2 - 900*k2*v3 - 900*k2*v4 + 140*k2 - 40*k3*u1 - 120*k3*u2 - 600*k3*u3 - 120*k3*v1 - 600*k3*v2 - 1800*k3*v3 - 1800*k3*v4 + 480*k3 + 6*u1*u2 + 30*u1*u3 + 6*u1*v1 + 30*u1*v2 + 90*u1*v3 + 90*u1*v4 - 3*u1 + 90*u2*u3 + 18*u2*v1 + 90*u2*v2 + 270*u2*v3 + 270*u2*v4 - 3*u2 + 90*u3*v1 + 450*u3*v2 + 1350*u3*v3 + 1350*u3*v4 + 165*u3 + 90*v1*v2 + 270*v1*v3 + 270*v1*v4 - 3*v1 + 1350*v2*v3 + 1350*v2*v4 + 165*v2 + 4050*v3*v4 + 1845*v3 + 1845*v4 + 4

In [23]:
expr_reduct_power == reduct_demension_expr

True

In [24]:
expr_ising = transfers_qubo2Ising(reduct_demension_expr)
solution = get_solution(expr_ising)
solution

{'k3': np.int8(-1),
 'v3': np.int8(-1),
 'v4': np.int8(-1),
 'k2': np.int8(-1),
 'u3': np.int8(-1),
 'v2': np.int8(-1),
 'u2': np.int8(1),
 'v1': np.int8(1),
 'u1': np.int8(1),
 'k1': np.int8(1)}

# test

In [25]:
def calcule_y(solution,expr_y,expr_k):
    dic_varialbe_value = {}

    for var in variables:
        valeur_ising = solution.get(var.name, np.int8(-1))

        if(valeur_ising == np.int8(1)):
            dic_varialbe_value[var] = int(1)
        else:
            dic_varialbe_value[var] = int(0)

    y_value = expr_y.subs(dic_varialbe_value)
    k_value = expr_k.subs(dic_varialbe_value)

    return (y_value.evalf(),k_value.evalf())

In [26]:
calcule_y(solution,expr_y,expr_k)

(3.00000000000000, 1.00000000000000)

## sum up fonction

In [27]:
def factorization(g,h,p):
    global variables, dictionaire_auxiliary_variable, res,expr_y,expr_k
    variables=[]
    dictionaire_auxiliary_variable={}
    res=1
    expr_y=sp.Add()
    expr_k=sp.Add()

    reduct_demension_expr = get_reduct_expression(g,h,p)
    expr_ising = transfers_qubo2Ising(reduct_demension_expr)
    solution = get_solution(expr_ising)
    print(solution)
    (y_val,k_val) = calcule_y(solution,expr_y,expr_k)
    y_int = int(y_val)
    k_int = int(k_val)
    print(f" k: {k_int}")
    print(f"The solution of {g} ^ y = {h} mod {p} is : y = {y_int}.")

In [35]:
g = 2
h = 3
p = 5
factorization(g,h,p)

linaire_expr : u1 + 3*u2 + 15*u3 + 3*v1 + 15*v2 + 45*v3 + 45*v4 + 1
 define_function : 25*k1**2 + 100*k1*k2 + 200*k1*k3 - 10*k1*u1 - 30*k1*u2 - 150*k1*u3 - 30*k1*v1 - 150*k1*v2 - 450*k1*v3 - 450*k1*v4 + 20*k1 + 100*k2**2 + 400*k2*k3 - 20*k2*u1 - 60*k2*u2 - 300*k2*u3 - 60*k2*v1 - 300*k2*v2 - 900*k2*v3 - 900*k2*v4 + 40*k2 + 400*k3**2 - 40*k3*u1 - 120*k3*u2 - 600*k3*u3 - 120*k3*v1 - 600*k3*v2 - 1800*k3*v3 - 1800*k3*v4 + 80*k3 + u1**2 + 6*u1*u2 + 30*u1*u3 + 6*u1*v1 + 30*u1*v2 + 90*u1*v3 + 90*u1*v4 - 4*u1 + 9*u2**2 + 90*u2*u3 + 18*u2*v1 + 90*u2*v2 + 270*u2*v3 + 270*u2*v4 - 12*u2 + 225*u3**2 + 90*u3*v1 + 450*u3*v2 + 1350*u3*v3 + 1350*u3*v4 - 60*u3 + 9*v1**2 + 90*v1*v2 + 270*v1*v3 + 270*v1*v4 - 12*v1 + 225*v2**2 + 1350*v2*v3 + 1350*v2*v4 - 60*v2 + 2025*v3**2 + 4050*v3*v4 - 180*v3 + 2025*v4**2 - 180*v4 + 4
{'k3': np.int8(1), 'v3': np.int8(-1), 'v4': np.int8(-1), 'k2': np.int8(-1), 'u3': np.int8(-1), 'v2': np.int8(1), 'u2': np.int8(1), 'v1': np.int8(1), 'u1': np.int8(1), 'k1': np.int8(-1)}
 k: 

In [29]:
g = 5
h = 4
p = 7
factorization(g,h,p)

linaire_expr : 4*u1 + 24*u2 + 624*u3 + 390624*u4 + 96*v1 + 5849985024*v10 + 23399940096*v11 + 2496*v2 + 14976*v3 + 1562496*v4 + 9374976*v5 + 243749376*v6 + 59904*v7 + 37499904*v8 + 974997504*v9 + 1
 define_function : 49*k1**2 + 196*k1*k2 + 392*k1*k3 + 784*k1*k4 - 56*k1*u1 - 336*k1*u2 - 8736*k1*u3 - 5468736*k1*u4 - 1344*k1*v1 - 81899790336*k1*v10 - 327599161344*k1*v11 - 34944*k1*v2 - 209664*k1*v3 - 21874944*k1*v4 - 131249664*k1*v5 - 3412491264*k1*v6 - 838656*k1*v7 - 524998656*k1*v8 - 13649965056*k1*v9 + 42*k1 + 196*k2**2 + 784*k2*k3 + 1568*k2*k4 - 112*k2*u1 - 672*k2*u2 - 17472*k2*u3 - 10937472*k2*u4 - 2688*k2*v1 - 163799580672*k2*v10 - 655198322688*k2*v11 - 69888*k2*v2 - 419328*k2*v3 - 43749888*k2*v4 - 262499328*k2*v5 - 6824982528*k2*v6 - 1677312*k2*v7 - 1049997312*k2*v8 - 27299930112*k2*v9 + 84*k2 + 784*k3**2 + 3136*k3*k4 - 224*k3*u1 - 1344*k3*u2 - 34944*k3*u3 - 21874944*k3*u4 - 5376*k3*v1 - 327599161344*k3*v10 - 1310396645376*k3*v11 - 139776*k3*v2 - 838656*k3*v3 - 87499776*k3*v4 - 524